In [2]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]


In [1]:
import os
import pandas as pd
import re
from pathlib import Path

# --- LINUX PATHS ---
root_dir = Path("/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset/inputs/images")
excel_path = Path("/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset/inputs/images/MetaData_Bsubt_KD_screen.xlsx")
output_file = Path("/media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset/inputs/metadata/index.csv")

# --- HELPER FUNCTIONS ---
def normalize_well_id(well):
    """Converts A1 -> A01, A11 -> A11"""
    match = re.match(r"([a-zA-Z])(\d+)", str(well).strip())
    if match:
        letter = match.group(1).upper()
        number = int(match.group(2))
        return f"{letter}{number:02d}"
    return str(well).strip()

def extract_gene(text):
    if pd.isna(text): return "Unknown"
    match = re.search(r'Gene target:\s*([\w-]+)', str(text))
    return match.group(1) if match else "Unknown"

# --- 1. LOAD ALL EXCEL SHEETS ---
print("Loading all Excel sheets...")
# sheet_name=None loads all sheets into a dictionary: {sheet_name: dataframe}
excel_dict = pd.read_excel(excel_path, sheet_name=None)

# Process each sheet into a treatment map
# Final structure: { 'PLATE_T2': {'A01': 'GeneA', 'A02': 'GeneB'}, 'PLATE_T3': {...} }
plate_treatment_maps = {}

for sheet_name, df in excel_dict.items():
    if 'Strain_name' in df.columns and 'Position_microscopy' in df.columns:
        df['Treatment'] = df['Strain_name'].apply(extract_gene)
        # Create a map for THIS specific sheet/plate
        current_map = dict(zip(
            df['Position_microscopy'].apply(normalize_well_id), 
            df['Treatment']
        ))
        plate_treatment_maps[sheet_name.strip()] = current_map

# --- 2. BUILD THE INDEX ---
all_rows = []
file_pattern = re.compile(r'Sample_(?P<well>[A-Z][0-9]+)_XY(?P<site>[0-9]+)_C(?P<chan>[0-9]+)')

print(f"Scanning directory: {root_dir}")

for plate_folder in os.listdir(root_dir):
    if "PLATE" not in plate_folder.upper():
        continue
    
    plate_path = root_dir / plate_folder
    if not plate_path.is_dir(): continue
    
    # Get the specific treatment map for this plate
    # We use .get() in case the folder name doesn't match the sheet name perfectly
    current_plate_map = plate_treatment_maps.get(plate_folder.strip(), {})
    
    if not current_plate_map:
        print(f"Warning: No Excel sheet found matching folder name '{plate_folder}'")
    
    for well_folder in os.listdir(plate_path):
        if not well_folder.startswith("Sample_"):
            continue
            
        well_path = plate_path / well_folder
        raw_well_id = well_folder.replace("Sample_", "")  # "A1"
        lookup_well_id = normalize_well_id(raw_well_id)    # "A01"
        
        sites = {} 
        for filename in os.listdir(well_path):
            if not filename.lower().endswith((".tiff", ".tif")):
                continue
            match = file_pattern.search(filename)
            if match:
                site_num = match.group('site')
                channel_num = f"C{match.group('chan')}"
                if site_num not in sites: sites[site_num] = {}
                sites[site_num][channel_num] = f"{plate_folder}/{well_folder}/{filename}"

        for site_id, channels in sites.items():
            row = {
                "Metadata_Plate": plate_folder,
                "Metadata_Well": raw_well_id,
                "Metadata_Site": int(site_id),
                "Treatment": current_plate_map.get(lookup_well_id, "Unknown"),
                "Replicate": 1 
            }
            row.update(channels)
            all_rows.append(row)

# --- 3. SAVE ---
if not all_rows:
    print("!!! NO IMAGES FOUND !!!")
else:
    df_final = pd.DataFrame(all_rows)
    channel_cols = sorted([c for c in df_final.columns if c.startswith("C")], 
                          key=lambda x: int(x[1:]) if x[1:].isdigit() else 0)
    final_cols = ["Metadata_Plate", "Metadata_Well", "Metadata_Site", "Treatment", "Replicate"] + channel_cols
    
    output_file.parent.mkdir(parents=True, exist_ok=True)
    df_final[final_cols].to_csv(output_file, index=False)
    
    unknown_count = len(df_final[df_final["Treatment"] == "Unknown"])
    print(f"DONE! Found {len(df_final)} sites.")
    if unknown_count > 0:
        print(f"Warning: {unknown_count} sites have 'Unknown' treatment. Check if sheet names match folder names exactly.")

Loading all Excel sheets...
Scanning directory: /media/arnout/Elements1/groteEdeepprofilerdingen/deepprofileroutputsetc/testset/inputs/images
DONE! Found 10 sites.


/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/arnout/miniconda3/envs/dp_analysis/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home